# Dzukou Pricing Toolkit – Colab Version
This notebook replicates the workflow of the Streamlit app. Run each section to add products, scrape prices, optimise pricing and generate a dashboard.

## Installation
Install the required packages.

In [ ]:
!pip install pandas plotly requests beautifulsoup4 selenium scipy scikit-optimize

## Repository Setup
Change directory to the cloned repository (update the path if needed).

In [ ]:
import os
if os.path.exists('/content/pricing'):
    os.chdir('/content/pricing')
!ls

## Add Products
The helper function below appends a product to the overview CSV and mapping file used by the optimiser.

In [ ]:
import pandas as pd
from pathlib import Path

OVERVIEW_CSV = Path('Dzukou_Pricing_Overview_With_Names - Copy.csv')
MAPPING_CSV = Path('product_data_mapping.csv')
DATA_DIR = Path('product_data')
DATA_DIR.mkdir(exist_ok=True)

def add_product(name, product_id, category, current_price, unit_cost, keywords=''):
    # update overview CSV
    if OVERVIEW_CSV.exists():
        df = pd.read_csv(OVERVIEW_CSV, encoding='cp1252')
    else:
        df = pd.DataFrame(columns=['Product Name','Product ID',' Current Price ',' Unit Cost'])
    df.loc[len(df)] = [name, product_id, current_price, unit_cost]
    df.to_csv(OVERVIEW_CSV, index=False, encoding='cp1252')
    # update mapping file
    if MAPPING_CSV.exists():
        mdf = pd.read_csv(MAPPING_CSV)
    else:
        mdf = pd.DataFrame(columns=['Product Name','Product ID','Data File'])
    data_file = DATA_DIR / f"{category.replace(' ','_').lower()}.csv"
    if not data_file.exists():
        data_file.write_text('category,store,product_name,price,search_term,store_url
')
    mdf.loc[len(mdf)] = [name, product_id, str(data_file)]
    mdf.to_csv(MAPPING_CSV, index=False)
    print('Added', name, '->', data_file)


Example call:

In [ ]:
add_product('Demo Sunglasses','SG999','Sunglasses',49.0,20.0,'demo, sunglasses')

## Scrape Competitor Prices
This step queries a few online stores for each category. It may take some time.

In [ ]:
from scraper import ProductScraper
sc = ProductScraper()
sc.scrape_all_stores()
sc.save_category_csvs()

## Price Optimisation

In [ ]:
import price_optimizer
price_optimizer.main()

## Inspect Recommendations

In [ ]:
import pandas as pd
rec = pd.read_csv('recommended_prices.csv')
rec.head()

## Build Dashboard

In [ ]:
import dashboard
df = dashboard.load_data()
dashboard.build_dashboard(df)
print('Dashboard saved to', dashboard.OUT_HTML)